In [1]:

# import the necessary packages
from skimage.segmentation import slic, quickshift
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import matplotlib.pyplot as plt
import argparse

import scipy
from scipy import ndimage, misc
import numpy as np
import os
import cv2
from PIL import Image
import numpy
import numpy as np
import torch
import pickle
import math

# Extract superpixels and create node features
import scipy.ndimage
from skimage.segmentation import slic
from scipy.spatial.distance import cdist
import sys

In [6]:
##All of this is for train data

import os # for iterating over images in a folder
# directory = '/Users/neginashouri/Desktop/Thesis/implimentation/CP_SmallerDataset_refined/Train/Type1'#the/directory/you/want/to/use'
# directory = '/Users/neginashouri/Desktop/Thesis/implimentation/CP_Dataset_refined/Train/Type_1'#the/directory/you/want/to/use'
directory = '/Users/neginashouri/Desktop/Thesis/implimentation/CP_Dataset_refined/Train_final/Type_3'
# directory = '/Users/neginashouri/Desktop/Thesis/implimentation/CP_Dataset_refined/Train/Type_3'
# out_path = '/Users/neginashouri/Desktop/Thesis/implimentation/create_superpixels/output'


x = 0 
image_list=[]
all_cp_sp_data = []

for filename in os.listdir(directory):
    
        x+=1
        
        if filename == '.DS_Store':
            continue

        # me: create the full input path and read the file
        input_path = os.path.join(directory, filename)
        
        #me: reading the images
        image = Image.open(input_path)
        
        #me: resizing the images into 1000*1000
        image = image.resize((1000, 1000))
        
        # me: convert the PIL image to numpy
        pix = np.array(image.getdata()).reshape(image.size[0], image.size[1],3) # convert the PIL image to numpy instead of # images = (image.numpy() / 255.)
#       print(pix.shape)
        
        pix_np = pix
#         print(pix_np.shape)
 
        image_list.append(image)
    

        img = pix_np.astype(np.double)  # 28x28 MNIST image #in My case 1000 * 1000
#         print("img shape",img.shape)

        # The number (n_segments) of superpixels returned by SLIC is usually smaller than requested, so we request more
        superpixels = slic(image, n_segments=100, compactness=15, multichannel= True, max_iter = 15,sigma = 10)
        sp_indices = np.unique(superpixels) # me: che adad hai darim bade superpixel kardan
        # print(sp_indices)
        n_sp = len(sp_indices)  # should be 74 with these parameters of slic #me: yani chanta daste bandi bara supper pixel
#         print(n_sp)


        sp_intensity = np.zeros((n_sp, 3), np.float32) # me: I added the 3rd dim (3)? is it correct?
        # print(sp_intensity)
        sp_coord = np.zeros((n_sp, 2), np.float32)  # row, col  # me: I added the 3rd dim (3)?is it correct?
        # print(sp_coord)


        for seg in sp_indices:
            mask = superpixels == seg # me: mask = True/False , 
#             print(mask.shape)


        ### intensity is the mean of each sp area and coord is the center of them. As they are the same on each layer(R,G,B), we are just calculating them on one layer. thay is why we used mask [:,:,1]
#             sp_intensity[seg] = np.concatenate([np.mean(img[mask]),np.mean(img[mask]),np.mean(img[mask])])
#             sp_intensity[seg] = np.mean(img[mask])
        ### /255 because the numbers in bench mark are 0< <1
            img1 = img[:,:,0]/255.0
#             print("shape img1",img1.shape)
            img1_ex = np.expand_dims(np.mean(img1[mask]), axis=0)
#             print("layer1 mean one seg:",img1_ex)
            
            img2 = img[:,:,1]/255.0
            img2_ex = np.expand_dims(np.mean(img2[mask]), axis=0)
#             print("layer2 mean one seg:",img2_ex)

            img3 = img[:,:,2]/255.0
            img3_ex = np.expand_dims(np.mean(img3[mask]), axis=0)
#             print("layer2 mean one seg:",img3_ex)
            
#             print("sp_intensity shape one seg:",np.concatenate([img1_ex,img2_ex,img3_ex]))
                  
            sp_intensity[seg] = np.concatenate([img1_ex,img2_ex,img3_ex])
#             print(sp_intensity.shape)
            sp_coord[seg] = np.array(scipy.ndimage.measurements.center_of_mass(mask)) #me: checked and all the layers have the same center # me:I think it is finding the location of the center of the mass#me : So the sp_coord is our nodes now
#             print(sp_coord.shape)
#             print("first layer sp",np.array(scipy.ndimage.measurements.center_of_mass(mask[:,:,0])))
#             print("2nd layer sp",np.array(scipy.ndimage.measurements.center_of_mass(mask[:,:,1])))
#             print("3rd layer sp",np.array(scipy.ndimage.measurements.center_of_mass(mask[:,:,2])))
#             if(np.random.rand() < 0.5):
#                 sp_intensity[seg] = math.nan
#                 sp_coord[seg] = (math.nan,math.nan)
#                 print("gharare berini")
#             if np.isnan(sp_intensity[seg]):
#                 print("begoo RIDI")
#             if np.isnan(sp_coord[seg][0]):
#                 print("begoo RIDI 2bare")
            
#         print("this is intensity",sp_intensity)
#         print("this is coord",sp_coord)
        cp_sp_data = (sp_intensity,sp_coord,sp_indices) ## this gives us the same format we want but shape is not ok and not sure if the whole list will be the same as what we want
#         print(cp_sp_data)
#         print(len(cp_sp_data))
        all_cp_sp_data.append(cp_sp_data) ## stack all the CPs
#         print(len(all_cp_sp_data))
        
            
        if(x%50 == 0):
            print(f'{x} is passed')
            
#         if (x==5):
        
    
##Me: in type{}_in_co_sp.data we are saving sp_intensity, sp_coord, sp_indices in 1 list, needs to be reshape to be aligned with other types sp shapes
        with open('type3_in_co_sp.pkl', 'wb') as filehandle:
            # store the data as binary data stream
            pickle.dump(all_cp_sp_data, filehandle)


#             break



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


50 is passed
100 is passed
150 is passed
200 is passed
250 is passed
300 is passed
350 is passed
400 is passed
450 is passed
500 is passed
550 is passed
600 is passed
650 is passed
700 is passed
750 is passed
800 is passed
850 is passed
900 is passed
950 is passed
1000 is passed
1050 is passed
1100 is passed
1150 is passed
1200 is passed
1250 is passed
1300 is passed
1350 is passed
1400 is passed
1450 is passed
1500 is passed
1550 is passed


In [7]:
##All of this is for test data

import os # for iterating over images in a folder
# directory = '/Users/neginashouri/Desktop/Thesis/implimentation/CP_Dataset_refined/Test/Type_1'
# directory = '/Users/neginashouri/Desktop/Thesis/implimentation/CP_Dataset_refined/Test/Type_2'
directory = '/Users/neginashouri/Desktop/Thesis/implimentation/CP_Dataset_refined/Test/Type_3'


x = 0 
image_list=[]
all_cp_sp_data = []

for filename in os.listdir(directory):
    
        x+=1

        if filename == '.DS_Store':
            continue
        # me: create the full input path and read the file
        input_path = os.path.join(directory, filename)
        
        
        #me: reading the images
        image = Image.open(input_path)
        
        #me: resizing the images into 1000*1000
        image = image.resize((1000, 1000))
        
        # me: convert the PIL image to numpy
        pix = np.array(image.getdata()).reshape(image.size[0], image.size[1],3) # convert the PIL image to numpy instead of # images = (image.numpy() / 255.)
#       print(pix.shape)
        
        pix_np = pix
#         print(pix_np.shape)
 
        image_list.append(image)
    

        img = pix_np.astype(np.double)  # 28x28 MNIST image #in My case 1000 * 1000
#         print("img shape",img.shape)

        # The number (n_segments) of superpixels returned by SLIC is usually smaller than requested, so we request more
        superpixels = slic(image, n_segments=100, compactness=15, multichannel= True, max_iter = 15,sigma = 10)
        sp_indices = np.unique(superpixels) # me: che adad hai darim bade superpixel kardan
        # print(sp_indices)
        n_sp = len(sp_indices)  # should be 74 with these parameters of slic #me: yani chanta daste bandi bara supper pixel
#         print(n_sp)


        sp_intensity = np.zeros((n_sp, 3), np.float32) # me: I added the 3rd dim (3)? is it correct?
        # print(sp_intensity)
        sp_coord = np.zeros((n_sp, 2), np.float32)  # row, col  # me: I added the 3rd dim (3)?is it correct?
        # print(sp_coord)


        for seg in sp_indices:
            mask = superpixels == seg # me: mask = True/False , 
#             print(mask.shape)


        ### /255 because the numbers in bench mark are 0< <1
            img1 = img[:,:,0]/255.0
#             print("shape img1",img1.shape)
            img1_ex = np.expand_dims(np.mean(img1[mask]), axis=0)
#             print("layer1 mean one seg:",img1_ex)
            
            img2 = img[:,:,1]/255.0
            img2_ex = np.expand_dims(np.mean(img2[mask]), axis=0)
#             print("layer2 mean one seg:",img2_ex)

            img3 = img[:,:,2]/255.0
            img3_ex = np.expand_dims(np.mean(img3[mask]), axis=0)
#             print("layer2 mean one seg:",img3_ex)
            
#             print("sp_intensity shape one seg:",np.concatenate([img1_ex,img2_ex,img3_ex]))
                  
            sp_intensity[seg] = np.concatenate([img1_ex,img2_ex,img3_ex])
#             print(sp_intensity.shape)
            sp_coord[seg] = np.array(scipy.ndimage.measurements.center_of_mass(mask)) #me: checked and all the layers have the same center # me:I think it is finding the location of the center of the mass#me : So the sp_coord is our nodes now

            

        cp_sp_data = (sp_intensity,sp_coord,sp_indices) ## this gives us the same format we want but shape is not ok and not sure if the whole list will be the same as what we want
#         print(cp_sp_data)
#         print(len(cp_sp_data))
        all_cp_sp_data.append(cp_sp_data) ## stack all the CPs
#         print(len(all_cp_sp_data))
        
            
        if(x%50 == 0):
            print(f'{x} is passed')
            
#         if (x==5):
        
    
##Me: in type{}_in_co_sp.data we are saving sp_intensity, sp_coord, sp_indices in 1 list, needs to be reshape to be aligned with other types sp shapes
        
#         for typ in range(1,4):
        with open('test_type3_in_co_sp.pkl', 'wb') as filehandle:
            # store the data as binary data stream
            pickle.dump(all_cp_sp_data, filehandle)


#             break



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


50 is passed
100 is passed
150 is passed
